In [416]:
import pandas as pd
import sqlite3
import json
import pyodbc
from datetime import datetime

In [417]:
students_file = 'data/students.txt'     
grades_file = 'data/api_grades.json'     
sqlite_db = 'data/courses.db'          

In [418]:
# 1. Extract 

In [419]:
students_df = pd.read_csv(students_file, sep='|')

In [420]:
students_df

,student_id,name,gender,city
0,1,Mohamed Ali,M,Alexandria
1,2,Amira Hassan,F,Cairo
2,3,Karim Saeed,M,Aswan
3,4,Sara Mohamed,F,Alexandria
4,5,Omar Adel,M,Cairo
5,6,Nour Youssef,F,Aswan
6,7,Tarek Ibrahim,M,Alexandria
7,8,Laila Samir,F,Cairo
8,9,Hassan Farouk,M,Aswan
9,10,Mona Fathy,F,Alexandria


In [421]:
conn = sqlite3.connect(sqlite_db)
cursor = conn.cursor()
cursor.execute("SELECT * FROM courses")
courses_data = cursor.fetchall()
courses_df = pd.DataFrame(courses_data, columns=['course_id', 'course_name','credits'])
cursor.close()
conn.close()

In [422]:
courses_df

,course_id,course_name,credits
0,201,Mathematics I,3
1,202,Physics I,3
2,203,Introduction to Programming,4
3,204,Databases,3
4,205,Software Engineering,3


In [423]:
with open(grades_file, 'r') as f:
    grades_data = json.load(f)
grades_data

[{'student_id': 1,
  'course_id': 201,
  'grade': 85,
  'attendance': 90,
  'date': '2024-01-15'},
 {'student_id': 2,
  'course_id': 202,
  'grade': 78,
  'attendance': 85,
  'date': '2024-01-18'},
 {'student_id': 3,
  'course_id': 203,
  'grade': 92,
  'attendance': 95,
  'date': '2024-02-02'},
 {'student_id': 4,
  'course_id': 204,
  'grade': 88,
  'attendance': 92,
  'date': '2024-02-10'},
 {'student_id': 5,
  'course_id': 205,
  'grade': 74,
  'attendance': 80,
  'date': '2024-03-05'},
 {'student_id': 6,
  'course_id': 201,
  'grade': 81,
  'attendance': 88,
  'date': '2024-03-12'},
 {'student_id': 7,
  'course_id': 202,
  'grade': 69,
  'attendance': 70,
  'date': '2024-04-01'},
 {'student_id': 8,
  'course_id': 203,
  'grade': 95,
  'attendance': 98,
  'date': '2024-04-07'},
 {'student_id': 9,
  'course_id': 204,
  'grade': 87,
  'attendance': 90,
  'date': '2024-05-03'},
 {'student_id': 10,
  'course_id': 205,
  'grade': 90,
  'attendance': 93,
  'date': '2024-05-20'}]

In [424]:
grades_df = pd.DataFrame(grades_data)

In [425]:
grades_df

,student_id,course_id,grade,attendance,date
0,1,201,85,90,2024-01-15
1,2,202,78,85,2024-01-18
2,3,203,92,95,2024-02-02
3,4,204,88,92,2024-02-10
4,5,205,74,80,2024-03-05
5,6,201,81,88,2024-03-12
6,7,202,69,70,2024-04-01
7,8,203,95,98,2024-04-07
8,9,204,87,90,2024-05-03
9,10,205,90,93,2024-05-20


In [426]:
# 2. Transform

In [427]:
grades_df['Date'] = pd.to_datetime(grades_df['date'])
grades_df['Date']

0   2024-01-15
1   2024-01-18
2   2024-02-02
3   2024-02-10
4   2024-03-05
5   2024-03-12
6   2024-04-01
7   2024-04-07
8   2024-05-03
9   2024-05-20
Name: Date, dtype: datetime64[ns]

In [428]:
dim_date = grades_df[['Date']].drop_duplicates().reset_index(drop=True)
dim_date

,Date
0,2024-01-15
1,2024-01-18
2,2024-02-02
3,2024-02-10
4,2024-03-05
5,2024-03-12
6,2024-04-01
7,2024-04-07
8,2024-05-03
9,2024-05-20


In [429]:
dim_date['DateKey'] = dim_date['Date'].dt.strftime('%Y%m%d').astype(int)
dim_date['DateKey'] 

0    20240115
1    20240118
2    20240202
3    20240210
4    20240305
5    20240312
6    20240401
7    20240407
8    20240503
9    20240520
Name: DateKey, dtype: int64

In [430]:
dim_date[['Year','Month','Day']] = dim_date['Date'].dt.strftime('%Y-%m-%d').str.split('-', expand=True).astype(int)


In [431]:
dim_date

,Date,DateKey,Year,Month,Day
0,2024-01-15,20240115,2024,1,15
1,2024-01-18,20240118,2024,1,18
2,2024-02-02,20240202,2024,2,2
3,2024-02-10,20240210,2024,2,10
4,2024-03-05,20240305,2024,3,5
5,2024-03-12,20240312,2024,3,12
6,2024-04-01,20240401,2024,4,1
7,2024-04-07,20240407,2024,4,7
8,2024-05-03,20240503,2024,5,3
9,2024-05-20,20240520,2024,5,20


In [432]:
fact_df = grades_df.merge(students_df[['student_id']], on='student_id', how='left')
fact_df = fact_df.merge(courses_df[['course_id']], on='course_id', how='left')
fact_df = fact_df.merge(dim_date[['Date', 'DateKey']], on='Date', how='left')
fact_df = fact_df[['student_id', 'course_id', 'grade', 'attendance', 'DateKey']]

fact_df

,student_id,course_id,grade,attendance,DateKey
0,1,201,85,90,20240115
1,2,202,78,85,20240118
2,3,203,92,95,20240202
3,4,204,88,92,20240210
4,5,205,74,80,20240305
5,6,201,81,88,20240312
6,7,202,69,70,20240401
7,8,203,95,98,20240407
8,9,204,87,90,20240503
9,10,205,90,93,20240520


In [433]:
fact_df = fact_df.astype({
    'student_id': int,
    'course_id': int,
    'grade': int,
    'attendance': int,
    'DateKey': int
})

In [434]:
# 3. Load

In [435]:
import pyodbc
print(pyodbc.drivers())


['SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)', 'SQL Server Native Client 11.0', 'ODBC Driver 11 for SQL Server', 'SQL Server Native Client RDA 11.0', 'ODBC Driver 17 for SQL Server']


In [436]:
import pyodbc

server = 'RETAJ\SQLEXPRESS'
database = 'StudentPerformanceDW'
driver = '{ODBC Driver 17 for SQL Server}'

conn = pyodbc.connect(
    f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
)

<>:3: SyntaxWarning: invalid escape sequence '\S'
<>:3: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Retaj\AppData\Local\Temp\ipykernel_28848\3011716349.py:3: SyntaxWarning: invalid escape sequence '\S'
  server = 'RETAJ\SQLEXPRESS'


In [437]:
students_df

,student_id,name,gender,city
0,1,Mohamed Ali,M,Alexandria
1,2,Amira Hassan,F,Cairo
2,3,Karim Saeed,M,Aswan
3,4,Sara Mohamed,F,Alexandria
4,5,Omar Adel,M,Cairo
5,6,Nour Youssef,F,Aswan
6,7,Tarek Ibrahim,M,Alexandria
7,8,Laila Samir,F,Cairo
8,9,Hassan Farouk,M,Aswan
9,10,Mona Fathy,F,Alexandria


In [438]:
students_data = students_df[['student_id', 'name', 'gender', 'city']].astype(str).values.tolist()
students_data

[['1', 'Mohamed Ali', 'M', 'Alexandria'],
 ['2', 'Amira Hassan', 'F', 'Cairo'],
 ['3', 'Karim Saeed', 'M', 'Aswan'],
 ['4', 'Sara Mohamed', 'F', 'Alexandria'],
 ['5', 'Omar Adel', 'M', 'Cairo'],
 ['6', 'Nour Youssef', 'F', 'Aswan'],
 ['7', 'Tarek Ibrahim', 'M', 'Alexandria'],
 ['8', 'Laila Samir', 'F', 'Cairo'],
 ['9', 'Hassan Farouk', 'M', 'Aswan'],
 ['10', 'Mona Fathy', 'F', 'Alexandria']]

In [439]:
courses_data = courses_df[['course_id', 'course_name', 'credits']].astype(object).values.tolist()
dates_data = dim_date[['DateKey', 'Date', 'Year', 'Month', 'Day']].copy()
dates_data['Date'] = dates_data['Date'].dt.date
dates_data = dates_data.values.tolist()
fact_data = fact_df.values.tolist()


In [440]:
cursor = conn.cursor()


# Insert into dim_student (includes City)
cursor.executemany(
    "INSERT INTO dim_student (StudentID, Name, Gender, City) VALUES (?, ?, ?, ?)",
    students_data
)

IntegrityError: ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Violation of PRIMARY KEY constraint 'PK__dim_stud__32C52A79168780FC'. Cannot insert duplicate key in object 'dbo.dim_student'. The duplicate key value is (1). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621)")

In [ ]:
conn.commit()

In [ ]:


# Insert into dim_course
cursor.executemany(
    "INSERT INTO dim_course (CourseID, CourseName, Credits) VALUES (?, ?, ?)",
    courses_data
)

# Insert into dim_date
cursor.executemany(
    "INSERT INTO dim_date (DateKey, FullDate, Year, Month, Day) VALUES (?, ?, ?, ?, ?)",
    dates_data
)

# Insert into fact_student_performance
cursor.executemany(
    """INSERT INTO fact_student_performance (StudentID, CourseID, Grade, Attendance, DateKey)
       VALUES (?, ?, ?, ?, ?)""",
    fact_data
)

conn.commit()
cursor.close()
conn.close()

print("✅ ETL Completed Successfully!")

IntegrityError: ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Violation of PRIMARY KEY constraint 'PK__dim_cour__C92D71872FF0462E'. Cannot insert duplicate key in object 'dbo.dim_course'. The duplicate key value is (201). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621)")